In [1]:
!pip install --quiet imgaug==0.2.6
!pip install --quiet scanpy statsmodels umap-learn loompy python-igraph

     |████████████████████████████████| 631 kB 3.2 MB/s 
     |████████████████████████████████| 2.0 MB 3.1 MB/s 
     |████████████████████████████████| 88 kB 6.0 MB/s 
     |████████████████████████████████| 4.8 MB 50.4 MB/s 
     |████████████████████████████████| 96 kB 4.7 MB/s 
     |████████████████████████████████| 11.2 MB 42.2 MB/s 
     |████████████████████████████████| 1.1 MB 50.3 MB/s 
     |████████████████████████████████| 944 kB 64.6 MB/s 
     |████████████████████████████████| 3.1 MB 44.1 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 


In [2]:
!pip install --quiet matplotlib==3.4

     |████████████████████████████████| 10.3 MB 2.7 MB/s 


In [3]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns 
import scanpy as sc
import scipy as sp
import numpy as np
import anndata
import pickle
from scipy import stats
from sklearn.metrics import cohen_kappa_score
from matplotlib import rcParams
#import pyscenic
import loompy

In [8]:
pt=sc.read_h5ad("/content/drive/MyDrive/KPMP/snRNA/analysis_06_2022/pt.aki.exclude.mixed.h5ad") #this is a PT dataset I created from seurat object, while excluding PT.mixed

In [9]:
pt=sc.AnnData(X=pt.raw.X,obs=pt.obs,var=pt.raw.var)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  """Entry point for launching an IPython kernel.


In [10]:
pt.var.index=pt.var['_index']

In [11]:
pt=pt[pt.obs["case.type.l1"]=="AKI",]

In [12]:
pt2=pt

In [13]:
pt2

View of AnnData object with n_obs × n_vars = 24528 × 34708
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'case.ident', 'case.type.l1', 'case.type.l2', 'RNA_snn_res.0.6', 'seurat_clusters', 'integrated_snn_res.0.5', 'case.ident2', 'integrated_snn_res.0.3', 'cluster.l1'
    var: '_index'

In [14]:
sc.pp.filter_genes(pt2, min_cells=5)

/usr/local/lib/python3.7/dist-packages/scanpy/preprocessing/_simple.py:251: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number


In [15]:
pt2

AnnData object with n_obs × n_vars = 24528 × 27468
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'case.ident', 'case.type.l1', 'case.type.l2', 'RNA_snn_res.0.6', 'seurat_clusters', 'integrated_snn_res.0.5', 'case.ident2', 'integrated_snn_res.0.3', 'cluster.l1'
    var: '_index', 'n_cells'

In [16]:
sc.pp.normalize_per_cell(pt2)
sc.pp.log1p(pt2)

In [19]:
sc.pp.highly_variable_genes(pt2, min_mean=0.0125, max_mean=3, min_disp=0.3) #
#sc.pl.highly_variable_genes(pt2)

In [20]:
hvg=pt2.var['highly_variable']
sum(hvg)

4757

In [ ]:
pt2=pt2[:,hvg]

In [ ]:
pt2.var.drop(["_index"],axis=1,inplace=True)

In [ ]:
pt2.var.index.rename('gene',inplace=True)

In [ ]:
pt2.var

,n_cells,highly_variable,means,dispersions,dispersions_norm
gene,,,,,
LINC01409,2226,True,0.128556,1.077362,0.412796
LINC01128,1049,True,0.046599,1.124260,0.578308
PLEKHN1,364,True,0.019877,1.204020,0.724435
ISG15,279,True,0.022749,1.653938,1.548725
AGRN,3418,True,0.173077,1.112542,0.504584
...,...,...,...,...,...
AC011751.1,242,True,0.019244,1.350420,0.992652
EIF1AY,747,True,0.032934,1.041284,0.426287
CTXN3,382,True,0.040761,1.793683,1.804749


In [ ]:
pt2.write_h5ad("pt_07152022_4757hvg.h5ad")

In [ ]:
  row_attrs = {
"Gene": np.array(pt2.var_names),
}
col_attrs = {
"CellID": np.array(pt2.obs_names),
"nGene": np.array(np.sum(pt2.X.transpose()>0, axis=0)).flatten(),
"nUMI": np.array(np.sum(pt2.X.transpose(),axis=0)).flatten(),
}
loompy.create("pt_07152022_4757hvg.loom",pt2.X.transpose(),row_attrs,
col_attrs)